In [74]:
import sqlite3
import csv
from math import sin, cos, acos, radians
import matplotlib.pyplot as plt
import seaborn
import numpy as np

In [21]:
# 距離計算（参考:http://qiita.com/s-wakaba/items/e12f2a575b6885579df7）
def latlng_to_xyz(lat, lng):
    rlat, rlng = radians(lat), radians(lng)
    coslat = cos(rlat)
    return coslat*cos(rlng), coslat*sin(rlng), sin(rlat)

def dist_on_sphere(lat1,lng1,lat2,lng2):
    radious = 6378.137
    xyz1, xyz2 = latlng_to_xyz(lat1,lng1), latlng_to_xyz(lat2,lng2)
    return acos(sum(x * y for x, y in zip(xyz1, xyz2)))*radious

In [82]:
# sqliteから選択した駅のレストラン情報（score,lat,lng）を抽出
def select_from_db(station_name):
    con_from = sqlite3.connect("./data/tokyo_all.sqlite3",timeout=60.0)
    rows = con_from.execute('select score,lat,lng from stores where nearest_station = ?',(station_name,))
    station_restaurants = []
    for row in rows:
        station_restaurants.append({'score': row[0], 'lat': row[1], 'lng': row[2]})
        print(row)
    return station_restaurants

In [40]:
def select_from_csv(staition_name):
    f = open('./data/yamanote_lat_lng.csv', 'r')
    reader = csv.reader(f)
    stations = {}
    for r in reader:
        stations[r[0] + '駅'] = {'lat': r[1], 'lng': r[2]}
    f.close()
    return stations[staition_name]

In [89]:
def get_dist_score(station_name):
    station_restaurants = select_from_db(station_name)
    station_place = select_from_csv(station_name)
    dist_score_list = []
    for rst in station_restaurants:
        if (not rst['score'])  or (rst['lng'] == 0.0):
            continue
        dist = dist_on_sphere(float(rst['lat']), float(rst['lng']), float(station_place['lat']), float(station_place['lng']))
        dist_score_list.append([ rst['score'], dist])
    return dist_score_list

In [97]:
dist_score_list = np.array( get_dist_score('池袋駅') )
plt.scatter(dist_score_list[: , 1], dist_score_list[: , 0])
plt.show()
# dist_score_list

('', 35.73132993281253, 139.7076079958225)
('', 35.73144327329832, 139.70871151006847)
('', 35.73033981254622, 139.71056251825888)
(3.03, 35.731213258879336, 139.7064775395231)
(3.15, 35.73493961651596, 139.71160388665666)
('', 35.73085523866497, 139.70511128492862)
('', 35.728572538668544, 139.71174408490097)
('', 35.73284452585782, 139.7094872144634)
(3.3, 35.730742369797916, 139.71076743280867)
('', 35.72893918206921, 139.71359927498858)
(3.0, 35.73208551646886, 139.7138435542827)
(3.02, 35.733566687380666, 139.7104343247786)
(3.0, 35.7296669386682, 139.7128633849113)
('', 35.72810425806587, 139.71283743225652)
('', 35.73106653866586, 139.70831648492853)
(3.54, 35.72857196671598, 139.71174389042977)
(3.31, 35.72857196671598, 139.71174389042977)
(3.09, 35.73322602950341, 139.71518872542097)
(3.55, 35.732368400147735, 139.7057358812665)
(3.44, 35.73279925548875, 139.7096341487894)
(3.01, 35.72993093866906, 139.7159824849113)
('', 35.73048600215689, 139.71595178617167)
(3.02, 35.730754